In [4]:
from pathlib import Path
from typing import Any
from digital_experiments.backends import CSVBackend
from digital_experiments.util import get_complete_config

def generate_id():
    return "42"

class Experiment:
    def __init__(self, callable):
        self.callable = callable
        self.backend = CSVBackend()
        self.directory = Path(".")

    @property
    def code(self):
        return self.callable.__code__
    
    def __call__(self, *args: Any, **kwds: Any) -> Any:
        _id = generate_id()
        config = get_complete_config(self.callable, args, kwds)
        result = self.callable(*args, **kwds)
        self.backend.save(config, result)


def experiment(callable):
    return Experiment(callable)


@experiment
def my_experiment():
    return 42

my_experiment()

42

In [7]:
from dataclasses import dataclass

@dataclass
class Observation:
    id: str
    config: dict
    result: Any
    metadata: dict = None


In [5]:
my_experiment

In [6]:
my_experiment.code

<code object my_experiment at 0x124a6db30, file "/var/folders/g9/k2dyt0ls1lzgtl01g3cm11r00000gr/T/ipykernel_3852/2820020350.py", line 20>

In [ ]:
import contextlib


__IDS = []
__RECORDING = False



def start_new():
    id = generate_id()
    __IDS.append(id)
    return id


def finished(id):
    assert __IDS[-1] == id
    __IDS.pop()


def current_id():
    return __IDS[-1]


@contextlib.contextmanager
def don_record():
    global __RECORDING
    __RECORDING = False
    yield
    __RECORDING = True


In [ ]:
def run(self, args, kwargs):
    if not control_center.should_record:
        return experiment(*args, **kwargs)
    
    id = control_center.start_new()
    config = get_complete_config(experiment, args, kwargs)

    self.metadata_collector.start()
    metadata, result = self._run_and_collect_metadata(experiment, args, kwargs)
    metadata = self.metadata_collector.stop()

    observation = Observation(id, config, result, metadata)
    
    control_center.finished(id)
    self._filesystem.save(observation)

    return result

# control flow:

call the experiment function with args and kwargs

1. if the file structure for saving experiments does not exist, create it (takes care of versioning etc.)
2. let the main controller know that this experiment is now running
3. start recording metadata
4. run the experiment
5. stop recording metadata
6. let the main controller know that this experiment is now finished
7. save everything to the file system
8. return the result


What does an observation have?
- id
- config
- result
- metadata
- artifacts

classes and their responsibilities:

Controller:
- knows about all experiments in the current session
- generates ids


Experiment:


In [21]:
from pathlib import Path

class CommandCenter:
    def __init__(self):
        self.known_experiments = {}
        self.running_experiments = []
    
    def register(self, experiment: Experiment):
        # experiments are uniquely identified by their code
        self.known_experiments[experiment.code] = experiment
    
    def starting(self, experiment: Experiment) -> str:
        id = self.generate_id()
        self.running_experiments.append((experiment, id))
        return id
        
    def finished(self, experiment: Experiment):
        current_id, current_experiment = self.running_experiments.pop()
        assert current_experiment == experiment

    def current_directory(self) -> Path:
        """
        get the directory assigned to the current experiment
        """

        id, experiment = self.running_experiments[-1]
        return experiment.file_system.directory_for(id)

    def generate_id(self) -> str:
        return 1


COMMAND_CENTER = CommandCenter()
current_directory = COMMAND_CENTER.current_directory

In [22]:
class FileSystem:
    def __init__(self, root: Path):
        self.root = root

    def directory_for(self, id: str) -> Path:
        return self.root / id

In [23]:
class Experiment:
    def __init__(self, callable):
        self.callable = callable
        self.file_system = FileSystem(Path("."))

    @property
    def code(self):
        # use inspect here
        return self.callable.__code__
    
    def __call__(self, *args: Any, **kwds: Any) -> Any:
        id = COMMAND_CENTER.starting(self)
        config = get_complete_config(self.callable, args, kwds)
        result = self.callable(*args, **kwds)
        COMMAND_CENTER.finished(self)
        return result
    
def experiment(callable):
    return Experiment(callable)

@experiment
def my_experiment():
    return 42

In [24]:
my_experiment()

NameError: name 'get_complete_config' is not defined